# Introduction

Using this Python notebook you will:

1.  Understand three Chicago datasets
2.  Load the three datasets into three tables in a SQLIte database
3.  Execute SQL queries to answer assignment questions


Execute the below code cell to install the required libraries


In [1]:
!pip install pandas
!pip install ipython-sql prettytable

import prettytable

prettytable.DEFAULT = 'DEFAULT'

### Store the datasets in database tables

To analyze the data using SQL, it first needs to be loaded into SQLite DB.
We will create three tables in as under:

1.  **CENSUS_DATA**
2.  **CHICAGO_PUBLIC_SCHOOLS**
3.  **CHICAGO_CRIME_DATA**


Load the `pandas` and `sqlite3` libraries and establish a connection to `FinalDB.db`


In [2]:
BASE_URL = "https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DB0201EN-SkillsNetwork/labs/FinalModule_Coursera_V5/data/Chicago"
QUERY = "utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkCoursesIBMDeveloperSkillsNetworkDB0201ENSkillsNetwork20127838-2021-01-01"
CENSUS_DATA_URL =               f"{BASE_URL}CensusData.csv?{QUERY}"
CHICAGO_PUBLIC_SCHOOLS_URL =    f"{BASE_URL}PublicSchools.csv?{QUERY}"
CHICAGO_CRIME_DATA_URL =        f"{BASE_URL}CrimeData.csv?{QUERY}"

Load the SQL magic module


In [3]:
%load_ext sql
%sql sqlite:///FinalDB.db

Use `Pandas` to load the data available in the links above to dataframes. Use these dataframes to load data on to the database `FinalDB.db` as required tables.


In [4]:
import pandas as pd

CENSUS_DATA = pd.read_csv(CENSUS_DATA_URL)
CHICAGO_PUBLIC_SCHOOLS = pd.read_csv(CHICAGO_PUBLIC_SCHOOLS_URL)
CHICAGO_CRIME_DATA = pd.read_csv(CHICAGO_CRIME_DATA_URL)

CHICAGO_PUBLIC_SCHOOLS.rename(columns={'Elementary, Middle, or High School': 'SCHOOL_TYPE'}, inplace=True)

Establish a connection between SQL magic module and the database `FinalDB.db`


In [5]:
import sqlite3

conn = sqlite3.connect('FinalDB.db')
CENSUS_DATA.to_sql("CENSUS_DATA", conn, if_exists='replace', index=False)
CHICAGO_PUBLIC_SCHOOLS.to_sql("CHICAGO_PUBLIC_SCHOOLS", conn, if_exists='replace', index=False)
CHICAGO_CRIME_DATA.to_sql("CHICAGO_CRIME_DATA", conn, if_exists='replace', index=False)

533

You can now proceed to the the following questions. Please note that a graded assignment will follow this lab and there will be a question on each of the problems stated below. It can be from the answer you received or the code you write for this problem. Therefore, please keep a note of both your codes as well as the response you generate.


## Problems

Now write and execute SQL queries to solve assignment problems

### Problem 1

##### Find the total number of crimes recorded in the CRIME table.


In [6]:
CHICAGO_CRIME_DATA.keys()

Index(['ID', 'CASE_NUMBER', 'DATE', 'BLOCK', 'IUCR', 'PRIMARY_TYPE',
       'DESCRIPTION', 'LOCATION_DESCRIPTION', 'ARREST', 'DOMESTIC', 'BEAT',
       'DISTRICT', 'WARD', 'COMMUNITY_AREA_NUMBER', 'FBICODE', 'X_COORDINATE',
       'Y_COORDINATE', 'YEAR', 'LATITUDE', 'LONGITUDE', 'LOCATION'],
      dtype='object')

In [7]:
%%sql

SELECT COUNT(CASE_NUMBER) as Total_Crimes FROM CHICAGO_CRIME_DATA

 * sqlite:///FinalDB.db
Done.


Total_Crimes
533


### Problem 2

##### List community area names and numbers with per capita income less than 11000.


In [8]:
CENSUS_DATA.keys()

Index(['COMMUNITY_AREA_NUMBER', 'COMMUNITY_AREA_NAME',
       'PERCENT_OF_HOUSING_CROWDED', 'PERCENT_HOUSEHOLDS_BELOW_POVERTY',
       'PERCENT_AGED_16__UNEMPLOYED',
       'PERCENT_AGED_25__WITHOUT_HIGH_SCHOOL_DIPLOMA',
       'PERCENT_AGED_UNDER_18_OR_OVER_64', 'PER_CAPITA_INCOME',
       'HARDSHIP_INDEX'],
      dtype='object')

In [9]:
%%sql

SELECT COMMUNITY_AREA_NUMBER, COMMUNITY_AREA_NAME FROM CENSUS_DATA WHERE PER_CAPITA_INCOME < 11000

 * sqlite:///FinalDB.db
Done.


COMMUNITY_AREA_NUMBER,COMMUNITY_AREA_NAME
26.0,West Garfield Park
30.0,South Lawndale
37.0,Fuller Park
54.0,Riverdale


### Problem 3

##### List all case numbers for crimes involving minors?(children are not considered minors for the purposes of crime analysis) 


In [10]:
%%sql

SELECT CASE_NUMBER, PRIMARY_TYPE, DESCRIPTION FROM CHICAGO_CRIME_DATA WHERE DESCRIPTION LIKE '%MINOR%';

 * sqlite:///FinalDB.db
Done.


CASE_NUMBER,PRIMARY_TYPE,DESCRIPTION
HL266884,LIQUOR LAW VIOLATION,SELL/GIVE/DEL LIQUOR TO MINOR
HK238408,LIQUOR LAW VIOLATION,ILLEGAL CONSUMPTION BY MINOR


### Problem 4

##### List all kidnapping crimes involving a child?


In [11]:
%%sql

SELECT CASE_NUMBER, PRIMARY_TYPE, DESCRIPTION FROM CHICAGO_CRIME_DATA WHERE PRIMARY_TYPE = 'KIDNAPPING' AND DESCRIPTION LIKE '%CHILD%';

 * sqlite:///FinalDB.db
Done.


CASE_NUMBER,PRIMARY_TYPE,DESCRIPTION
HN144152,KIDNAPPING,CHILD ABDUCTION/STRANGER


### Problem 5

##### List the kind of crimes that were recorded at schools. (No repetitions)


In [12]:
%%sql

SELECT DISTINCT ccd.PRIMARY_TYPE
FROM CHICAGO_CRIME_DATA ccd
JOIN CHICAGO_PUBLIC_SCHOOLS cps
  ON ccd.COMMUNITY_AREA_NUMBER = cps.COMMUNITY_AREA_NUMBER
WHERE ccd.LOCATION_DESCRIPTION LIKE '%SCHOOL%';

 * sqlite:///FinalDB.db
Done.


PRIMARY_TYPE
BATTERY
CRIMINAL DAMAGE
NARCOTICS
ASSAULT
CRIMINAL TRESPASS


### Problem 6

##### List the type of schools along with the average safety score for each type.


In [13]:
%%sql

SELECT 
  SCHOOL_TYPE,
  AVG(SAFETY_SCORE) AS AVERAGE_SAFETY_SCORE
FROM CHICAGO_PUBLIC_SCHOOLS
WHERE SAFETY_SCORE IS NOT NULL
GROUP BY SCHOOL_TYPE;


 * sqlite:///FinalDB.db
Done.


SCHOOL_TYPE,AVERAGE_SAFETY_SCORE
ES,49.52038369304557
HS,49.62352941176471
MS,48.0


### Problem 7

##### List 5 community areas with highest % of households below poverty line


In [14]:
%%sql

SELECT COMMUNITY_AREA_NAME, PERCENT_HOUSEHOLDS_BELOW_POVERTY FROM CENSUS_DATA 
ORDER BY PERCENT_HOUSEHOLDS_BELOW_POVERTY DESC LIMIT 5;

 * sqlite:///FinalDB.db
Done.


COMMUNITY_AREA_NAME,PERCENT_HOUSEHOLDS_BELOW_POVERTY
Riverdale,56.5
Fuller Park,51.2
Englewood,46.6
North Lawndale,43.1
East Garfield Park,42.4


### Problem 8

##### Which community area is most crime prone? Display the coumminty area number only.


In [15]:
%%sql

SELECT COMMUNITY_AREA_NUMBER, COUNT(*) AS CRIME_COUNT
FROM CHICAGO_CRIME_DATA
GROUP BY COMMUNITY_AREA_NUMBER
ORDER BY CRIME_COUNT DESC
LIMIT 1;


 * sqlite:///FinalDB.db
Done.


COMMUNITY_AREA_NUMBER,CRIME_COUNT
25.0,43


### Problem 9

##### Use a sub-query to find the name of the community area with highest hardship index


In [16]:
%%sql

SELECT COMMUNITY_AREA_NAME
FROM CENSUS_DATA
WHERE HARDSHIP_INDEX IN (
    SELECT MAX(HARDSHIP_INDEX)
    FROM CENSUS_DATA
);

 * sqlite:///FinalDB.db
Done.


COMMUNITY_AREA_NAME
Riverdale


### Problem 10

##### Use a sub-query to determine the Community Area Name with most number of crimes?


In [17]:
%%sql

SELECT COMMUNITY_AREA_NAME
FROM CENSUS_DATA
WHERE COMMUNITY_AREA_NUMBER = (
    SELECT COMMUNITY_AREA_NUMBER
    FROM CHICAGO_CRIME_DATA
    GROUP BY COMMUNITY_AREA_NUMBER
    ORDER BY COUNT(*) DESC
    LIMIT 1
);


 * sqlite:///FinalDB.db
Done.


COMMUNITY_AREA_NAME
Austin
